In [1]:
import openreview
from deepdiff import DeepDiff
from pprint import pprint
from __future__ import print_function

guest_client = openreview.Client(baseurl="https://api.openreview.net")

def sort_by_tcdate(notes):
    if type(notes[0]) is dict:
        return list(sorted(notes, key=lambda x: x['tcdate']))
    else:
        return list(sorted(notes, key=lambda x: x.tcdate))

def print_metadata(note):
    if type(note) is dict:
        print(
            note['id'], note['forum'], 
            note['original'] if 'original' in note else None, 
            note['referent'] if 'referent' in note else None, 
            note['cdate'], note['tcdate'], note['tmdate'], note['invitation']
        )
    else:
        print(
            note.id, note.forum, note.original, note.referent,
            note.cdate, note.tcdate, note.tmdate, note.invitation
        )

In [2]:
VENUE = "ICLR.cc/2019/Conference"

In [3]:
# get all blind submissions
forum_notes = list(openreview.tools.iterget_notes(
    guest_client, invitation=f"{VENUE}/-/Blind_Submission", sort='tcdate',
))
for forum_note in forum_notes:
    assert forum_note.id == forum_note.forum and forum_note.id != forum_note.original
print('#BlindSubmission:', len(forum_notes))

#BlindSubmission: 1419


### Referent submission and comment notes

In [4]:
# get a blind submission
forum_note = forum_notes[0]
print('id forum original referent cdate tcdate tmdate invitation')
print_metadata(forum_note)
print(forum_note)

id forum original referent cdate tcdate tmdate invitation
rJl0r3R9KX rJl0r3R9KX HkeaLA39Ym None 1538088006157 1538088006157 1644493216632 ICLR.cc/2019/Conference/-/Blind_Submission
{'cdate': 1538088006157,
 'content': {'TL;DR': 'A practical and provably guaranteed approach for '
                      'training efficiently classifiers in the presence of '
                      'label shifts between Source and Target data sets',
             '_bibtex': '@inproceedings{\n'
                        'azizzadenesheli2018regularized,\n'
                        'title={Regularized Learning for  Domain Adaptation '
                        'under Label Shifts},\n'
                        'author={Kamyar Azizzadenesheli and Anqi Liu and Fanny '
                        'Yang and Animashree Anandkumar},\n'
                        'booktitle={International Conference on Learning '
                        'Representations},\n'
                        'year={2019},\n'
                        'url={http

In [5]:
# get the referent submission (forum) note and comment notes for this blind submission
this_forum_notes = sort_by_tcdate(list(openreview.tools.iterget_notes(guest_client, forum=forum_note.id)))
print('#NotesThisBlindSubmissionForum', len(this_forum_notes))

print('id forum original referent cdate tcdate tmdate invitation')
for note in this_forum_notes:
    print_metadata(note)
    if note.id == forum_note.id:
        print(DeepDiff(note, forum_note, ignore_order=True))
    else:
        comment_note = note
assert forum_note.id in [note.id for note in this_forum_notes]

#NotesThisBlindSubmissionForum 11
id forum original referent cdate tcdate tmdate invitation
rJl0r3R9KX rJl0r3R9KX HkeaLA39Ym None 1538088006157 1538088006157 1644493216632 ICLR.cc/2019/Conference/-/Blind_Submission
{}
rkl-PP8d2Q rJl0r3R9KX None None 1541068633167 1541068633167 1541533006339 ICLR.cc/2019/Conference/-/Paper1592/Official_Review
SyxJIBjuhm rJl0r3R9KX None None 1541088583253 1541088583253 1541533006545 ICLR.cc/2019/Conference/-/Paper1592/Official_Review
r1ghyrDc2m rJl0r3R9KX None None 1541203171721 1541203171721 1543160898185 ICLR.cc/2019/Conference/-/Paper1592/Official_Review
BJg59Q0iT7 rJl0r3R9KX None None 1542345617852 1542345617852 1542368233221 ICLR.cc/2019/Conference/-/Paper1592/Official_Comment
Hkldv4Aj6X rJl0r3R9KX None None 1542345824485 1542345824485 1542345824485 ICLR.cc/2019/Conference/-/Paper1592/Official_Comment
BkeG9kNh6X rJl0r3R9KX None None 1542369162421 1542369162421 1542369457450 ICLR.cc/2019/Conference/-/Paper1592/Official_Comment
rygX0-Ehp7 rJl0r3R9KX N

### Referent submission and its references

In [6]:
print("forum id\t", forum_note.id)
print("original id\t", forum_note.original)

forum id	 rJl0r3R9KX
original id	 HkeaLA39Ym


In [7]:
# original submission and its revisions
submission_references_oT = sort_by_tcdate(guest_client.get_all_references(referent=forum_note.id, original=True))
print('#References:', len(submission_references_oT))

has_original = False
print('id forum original referent cdate tcdate tmdate invitation')
for ref in submission_references_oT:
    assert ref.original == forum_note.original
    print_metadata(ref)
    if ref.id == forum_note.original:
        print("above is the original submission note")
        has_original = True
    # TODO check that the original has the same contents as the forum
    # TODO only need one of this and the forum note
if not has_original:
    print("MISSING THE ORIGINAL NOTE")

#References: 8
id forum original referent cdate tcdate tmdate invitation
BJpIR25YX rJl0r3R9KX HkeaLA39Ym rJl0r3R9KX 1538088006157 1538080341168 1539203907586 ICLR.cc/2019/Conference/-/Blind_Submission
Sk_lKX9Tm rJl0r3R9KX HkeaLA39Ym rJl0r3R9KX 1538088006157 1542236399679 1542236399679 ICLR.cc/2019/Conference/-/Blind_Submission
ry79jm5pQ rJl0r3R9KX HkeaLA39Ym rJl0r3R9KX 1538088006157 1542237067058 1542237067058 ICLR.cc/2019/Conference/-/Blind_Submission
ByCx70sam rJl0r3R9KX HkeaLA39Ym rJl0r3R9KX 1538088006157 1542345461793 1542345461793 ICLR.cc/2019/Conference/-/Blind_Submission
BJVDsE367 rJl0r3R9KX HkeaLA39Ym rJl0r3R9KX 1538088006157 1542372187604 1542372187604 ICLR.cc/2019/Conference/-/Blind_Submission
rkXUYERSN rJl0r3R9KX HkeaLA39Ym rJl0r3R9KX 1538088006157 1550891339090 1550891339090 ICLR.cc/2019/Conference/-/Blind_Submission
BkaTYNRS4 rJl0r3R9KX HkeaLA39Ym rJl0r3R9KX 1538088006157 1550891460918 1550891460918 ICLR.cc/2019/Conference/-/Blind_Submission
rtZtjY_fk9 rJl0r3R9KX HkeaLA39Y

In [8]:
# this seems to return one of the above revisions, which is Code
guest_client.get_all_references(referent=forum_note.original, original=True)

[Note(id = 'rtZtjY_fk9',original = None,number = 626,cdate = 1644493216632,mdate = 1644493216632,tcdate = 1644493216632,tmdate = 1644493216632,ddate = None,content = {'code': '[![Papers with Code](/images/pwc_icon.svg) 2 community implementations](https://paperswithcode.com/paper/?openreview=rJl0r3R9KX)', 'data': '[CIFAR-10](https://paperswithcode.com/dataset/cifar-10)'},forum = 'HkeaLA39Ym',referent = 'HkeaLA39Ym',invitation = 'PapersWithCode.com/-/ICLR_2019_Code',replyto = None,readers = ['everyone'],nonreaders = [],signatures = ['PapersWithCode.com'],writers = [],details = {})]

In [9]:
guest_client.get_all_references(referent=forum_note.original, original=False)

[Note(id = 'rtZtjY_fk9',original = None,number = 626,cdate = 1644493216632,mdate = 1644493216632,tcdate = 1644493216632,tmdate = 1644493216632,ddate = None,content = {'code': '[![Papers with Code](/images/pwc_icon.svg) 2 community implementations](https://paperswithcode.com/paper/?openreview=rJl0r3R9KX)', 'data': '[CIFAR-10](https://paperswithcode.com/dataset/cifar-10)'},forum = 'HkeaLA39Ym',referent = 'HkeaLA39Ym',invitation = 'PapersWithCode.com/-/ICLR_2019_Code',replyto = None,readers = ['everyone'],nonreaders = [],signatures = ['PapersWithCode.com'],writers = [],details = None)]

In [10]:
print(forum_note)

{'cdate': 1538088006157,
 'content': {'TL;DR': 'A practical and provably guaranteed approach for '
                      'training efficiently classifiers in the presence of '
                      'label shifts between Source and Target data sets',
             '_bibtex': '@inproceedings{\n'
                        'azizzadenesheli2018regularized,\n'
                        'title={Regularized Learning for  Domain Adaptation '
                        'under Label Shifts},\n'
                        'author={Kamyar Azizzadenesheli and Anqi Liu and Fanny '
                        'Yang and Animashree Anandkumar},\n'
                        'booktitle={International Conference on Learning '
                        'Representations},\n'
                        'year={2019},\n'
                        'url={https://openreview.net/forum?id=rJl0r3R9KX},\n'
                        '}',
             'abstract': 'We propose Regularized Learning under Label shifts '
                         '(RL

In [11]:
print(submission_references_oT[0]) # original

{'cdate': 1538088006157,
 'content': {'TL;DR': 'A practical and provably guaranteed approach   for '
                      'efficiently training a classifier when there is a label '
                      'shift between Source and Target',
             '_bibtex': '@inproceedings{\n'
                        'azizzadenesheli2018regularized,\n'
                        'title={Regularized Learning for  Domain Adaptation '
                        'under Label Shifts},\n'
                        'author={Kamyar Azizzadenesheli and Anqi Liu and Fanny '
                        'Yang and Animashree Anandkumar},\n'
                        'booktitle={International Conference on Learning '
                        'Representations},\n'
                        'year={2019},\n'
                        'url={https://openreview.net/forum?id=rJl0r3R9KX},\n'
                        '}',
             'abstract': 'We propose Regularized Learning under Label shifts '
                         '(RLLS), a prin

In [12]:
print(submission_references_oT[1]) # revision

{'cdate': 1538088006157,
 'content': {'TL;DR': 'A practical and provably guaranteed approach   for '
                      'efficiently training a classifier when there is a label '
                      'shift between Source and Target',
             '_bibtex': '@inproceedings{\n'
                        'azizzadenesheli2018regularized,\n'
                        'title={Regularized Learning for  Domain Adaptation '
                        'under Label Shifts},\n'
                        'author={Kamyar Azizzadenesheli and Anqi Liu and Fanny '
                        'Yang and Animashree Anandkumar},\n'
                        'booktitle={International Conference on Learning '
                        'Representations},\n'
                        'year={2019},\n'
                        'url={https://openreview.net/forum?id=rJl0r3R9KX},\n'
                        '}',
             'abstract': 'We propose Regularized Learning under Label shifts '
                         '(RLLS), a prin

### A referent comment note and its references

In [13]:
comment_references_oT = sort_by_tcdate(guest_client.get_all_references(referent=comment_note.id, original=True))
print('#References:', len(comment_references_oT))
print('id forum original referent cdate tcdate tmdate invitation')
for ref in comment_references_oT:
    print_metadata(ref)

#References: 1
id forum original referent cdate tcdate tmdate invitation
ByQbcQrxN rJl0r3R9KX None HJlQWqQreN 1545054715049 1545054715049 1545354483441 ICLR.cc/2019/Conference/-/Paper1592/Meta_Review


In [14]:
comment_references_oF = sort_by_tcdate(guest_client.get_all_references(referent=comment_note.id, original=False))
print('#References:', len(comment_references_oF))
print('id forum original referent cdate tcdate tmdate invitation')
for ref in comment_references_oF:
    print_metadata(ref)

#References: 1
id forum original referent cdate tcdate tmdate invitation
ByQbcQrxN rJl0r3R9KX None HJlQWqQreN 1545054715049 1545054715049 1545354483441 ICLR.cc/2019/Conference/-/Paper1592/Meta_Review


In [15]:
print(comment_note)

{'cdate': 1545054715049,
 'content': {'confidence': '5: The area chair is absolutely certain',
             'metareview': 'The paper gives a novel algorithm for transfer '
                           'learning with label distribution shift with '
                           'provably guarantees. As the reviewers pointed out, '
                           'the pros include: 1) a solid and motivated '
                           'algorithm for a understudied problem 2) the '
                           'algorithm is implemented empirically and gives '
                           'good performance. The drawback includes '
                           'incomplete/unclear comparison with previous work. '
                           'The authors claimed that the code of the previous '
                           'work cannot be completed within a reasonable '
                           'amount of time. The AC decided that the paper '
                           'could be accepted without such a compari

In [16]:
print(comment_references_oT[0])

{'cdate': 1545054715049,
 'content': {'confidence': '5: The area chair is absolutely certain',
             'metareview': 'The paper gives a novel algorithm for transfer '
                           'learning with label distribution shift with '
                           'provably guarantees. As the reviewers pointed out, '
                           'the pros include: 1) a solid and motivated '
                           'algorithm for a understudied problem 2) the '
                           'algorithm is implemented empirically and gives '
                           'good performance. The drawback includes '
                           'incomplete/unclear comparison with previous work. '
                           'The authors claimed that the code of the previous '
                           'work cannot be completed within a reasonable '
                           'amount of time. The AC decided that the paper '
                           'could be accepted without such a compari

In [17]:
print(comment_references_oF[0])

{'cdate': 1545054715049,
 'content': {'confidence': '5: The area chair is absolutely certain',
             'metareview': 'The paper gives a novel algorithm for transfer '
                           'learning with label distribution shift with '
                           'provably guarantees. As the reviewers pointed out, '
                           'the pros include: 1) a solid and motivated '
                           'algorithm for a understudied problem 2) the '
                           'algorithm is implemented empirically and gives '
                           'good performance. The drawback includes '
                           'incomplete/unclear comparison with previous work. '
                           'The authors claimed that the code of the previous '
                           'work cannot be completed within a reasonable '
                           'amount of time. The AC decided that the paper '
                           'could be accepted without such a compari

## Other exploration

### Get notes with direct or all replies

In [18]:
# get all blind submissions, each with its direct replies
forum_notes_dr = list(openreview.tools.iterget_notes(
    guest_client, invitation=f"{VENUE}/-/Blind_Submission", sort='tcdate',
    details='directReplies'
))
for forum_note in forum_notes:
    assert forum_note.id == forum_note.forum and forum_note.id != forum_note.original
print('#BlindSubmission:', len(forum_notes))

#BlindSubmission: 1419


In [19]:
# get all blind submissions, each with its replies
forum_notes_r = list(openreview.tools.iterget_notes(
    guest_client, invitation=f"{VENUE}/-/Blind_Submission", sort='tcdate',
    details='replies'
))
for forum_note in forum_notes:
    assert forum_note.id == forum_note.forum and forum_note.id != forum_note.original
print('#BlindSubmission:', len(forum_notes))

#BlindSubmission: 1419


In [20]:
# the first blind submission with direct replies
forum_note = forum_notes_dr[0]
print('#directReplies:', len(forum_note.details['directReplies']))
for reply in sort_by_tcdate(forum_note.details['directReplies']):
    print_metadata(reply)
print()
print_metadata(forum_note)
print(forum_note)

#directReplies: 5
rkl-PP8d2Q rJl0r3R9KX None None 1541068633167 1541068633167 1541533006339 ICLR.cc/2019/Conference/-/Paper1592/Official_Review
SyxJIBjuhm rJl0r3R9KX None None 1541088583253 1541088583253 1541533006545 ICLR.cc/2019/Conference/-/Paper1592/Official_Review
r1ghyrDc2m rJl0r3R9KX None None 1541203171721 1541203171721 1543160898185 ICLR.cc/2019/Conference/-/Paper1592/Official_Review
BJg59Q0iT7 rJl0r3R9KX None None 1542345617852 1542345617852 1542368233221 ICLR.cc/2019/Conference/-/Paper1592/Official_Comment
HJlQWqQreN rJl0r3R9KX None None 1545054715049 1545054715049 1545354483441 ICLR.cc/2019/Conference/-/Paper1592/Meta_Review

rJl0r3R9KX rJl0r3R9KX HkeaLA39Ym None 1538088006157 1538088006157 1644493216632 ICLR.cc/2019/Conference/-/Blind_Submission
{'cdate': 1538088006157,
 'content': {'TL;DR': 'A practical and provably guaranteed approach for '
                      'training efficiently classifiers in the presence of '
                      'label shifts between Source and 

In [21]:
# the first blind submission with all replies
forum_note = forum_notes_r[0]
print('#replies:', len(forum_note.details['replies']))
for reply in sort_by_tcdate(forum_note.details['replies']):
    print_metadata(reply)
print()
print_metadata(forum_note)
print(forum_note)

#replies: 10
rkl-PP8d2Q rJl0r3R9KX None None 1541068633167 1541068633167 1541533006339 ICLR.cc/2019/Conference/-/Paper1592/Official_Review
SyxJIBjuhm rJl0r3R9KX None None 1541088583253 1541088583253 1541533006545 ICLR.cc/2019/Conference/-/Paper1592/Official_Review
r1ghyrDc2m rJl0r3R9KX None None 1541203171721 1541203171721 1543160898185 ICLR.cc/2019/Conference/-/Paper1592/Official_Review
BJg59Q0iT7 rJl0r3R9KX None None 1542345617852 1542345617852 1542368233221 ICLR.cc/2019/Conference/-/Paper1592/Official_Comment
Hkldv4Aj6X rJl0r3R9KX None None 1542345824485 1542345824485 1542345824485 ICLR.cc/2019/Conference/-/Paper1592/Official_Comment
BkeG9kNh6X rJl0r3R9KX None None 1542369162421 1542369162421 1542369457450 ICLR.cc/2019/Conference/-/Paper1592/Official_Comment
rygX0-Ehp7 rJl0r3R9KX None None 1542369739111 1542369739111 1542369793530 ICLR.cc/2019/Conference/-/Paper1592/Official_Comment
BJe5ulInCQ rJl0r3R9KX None None 1543426162395 1543426162395 1543426162395 ICLR.cc/2019/Conference/-/P

### Get references with other settings

In [24]:
# referent=forum_note.id, original=False
# -> for ICLR 2018 this returns the submission/forum note with little content
# -> here it returns some new note with little content
submission_references_oF = sort_by_tcdate(guest_client.get_all_references(referent=forum_note.id, original=False))
print('#References:', len(submission_references_oF))
assert len(submission_references_oF) == 1

print('id forum original referent cdate tcdate tmdate invitation')
for ref in submission_references_oF:
    print_metadata(ref)
    # assert ref.id == forum_note.id
    assert ref.original == forum_note.original

#References: 1
id forum original referent cdate tcdate tmdate invitation
SJCr209tm rJl0r3R9KX HkeaLA39Ym rJl0r3R9KX 1538088006157 1538088006157 1545355433068 ICLR.cc/2019/Conference/-/Blind_Submission


In [25]:
print(submission_references_oF[0])

{'cdate': 1538088006157,
 'content': {'_bibtex': '@inproceedings{\n'
                        'azizzadenesheli2018regularized,\n'
                        'title={Regularized Learning for  Domain Adaptation '
                        'under Label Shifts},\n'
                        'author={Kamyar Azizzadenesheli and Anqi Liu and Fanny '
                        'Yang and Animashree Anandkumar},\n'
                        'booktitle={International Conference on Learning '
                        'Representations},\n'
                        'year={2019},\n'
                        'url={https://openreview.net/forum?id=rJl0r3R9KX},\n'
                        '}'},
 'ddate': None,
 'details': None,
 'forum': 'rJl0r3R9KX',
 'id': 'SJCr209tm',
 'invitation': 'ICLR.cc/2019/Conference/-/Blind_Submission',
 'mdate': None,
 'nonreaders': [],
 'number': 1592,
 'original': 'HkeaLA39Ym',
 'readers': ['everyone'],
 'referent': 'rJl0r3R9KX',
 'replyto': None,
 'signatures': ['ICLR.cc/2019/Conference'],

### Seemingly useful but actually confusing

In [ ]:
# get venue-specific event types
# the problem is that, unexpectedly, this doesn't return Official_Review
invitations = openreview.tools.iterget_invitations(
        guest_client, regex=f'{VENUE}.*'
)
event_types = set()
for invitation in invitations:
    event_types.add(invitation.id.split('/')[-1])
print(event_types)